<a href="https://colab.research.google.com/github/Chikuji/AzureML/blob/master/2020_09_22_04_TrabalharComComputacaoAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Objetivos de aprendizado

- Trabalhe com ambientes
-Trabalho com alvos de computação

# Criação de ambientes

Criação de um ambiente a partir de um arquivo de especificação.

Você pode usar um arquivo de especificação Conda ou pip para definir os pacotes necessários em um ambiente Python e usá-lo para criar um objeto Environment .

Por exemplo, você pode salvar as seguintes configurações do Conda em um arquivo denominado conda.yml

In [ ]:
name: py_env
dependencies:
  - numpy
  - pandas
  - scikit-learn
  - pip:
    - azureml-defaults

Em seguida, você pode usar o seguinte código para criar um ambiente do Azure Machine Learning a partir do arquivo de especificação salvo:

In [ ]:
from azureml.core import Environment

env = Environment.from_conda_specification(name='training_environment',
                                           file_path='./conda.yml')

Criação de um ambiente a partir de um ambiente Conda existente

In [ ]:
from azureml.core import Environment

env = Environment.from_existing_conda_environment(name='training_environment',
                                                  conda_environment_name='py_env')

Criação de um ambiente especificando pacotes

In [ ]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

env = Environment('training_environment')
deps = CondaDependencies.create(conda_packages=['scikit-learn','pandas','numpy'],
                                pip_packages=['azureml-defaults'])
env.python.conda_dependencies = deps

# Registrando e reutilizando ambientes

Registrando um ambiente

In [ ]:
env.register(workspace=ws)

Você pode visualizar os ambientes registrados em seu espaço de trabalho assim:

In [ ]:
from azureml.core import Environment

env_names = Environment.list(workspace=ws)
for env_name in env_names:
    print('Name:',env_name)

Recuperando e usando um ambiente

In [ ]:
from azureml.core import Environment
from azureml.train.estimator import Estimator

training_env = Environment.get(workspace=ws, name='training_environment')
estimator = Estimator(source_directory='experiment_folder'
                      entry_script='training_script.py',
                      compute_target='local',
                      environment_definition=training_env)

# Introdução aos alvos computacionais

No Azure Machine Learning, você pode aproveitar esse princípio definindo destinos de computação que:

- Inicie sob demanda e pare automaticamente quando não for mais necessário.
-Escale automaticamente com base nas necessidades de processamento da carga de trabalho.

# Tipos de computação

Computação local

Clusters de computação

Clusters de inferência

Computação anexada

# Criação de um destino de computação gerenciado com o SDK

Para criar um cluster de computação do Azure Machine Learning, use a classe azureml.core.compute.ComputeTarget e a classe AmlCompute , como este:

In [ ]:
from azureml.core import Workspace
from azureml.core.compute import ComputeTarget, AmlCompute

# Load the workspace from the saved config file
ws = Workspace.from_config()

# Specify a name for the compute (unique within the workspace)
compute_name = 'aml-cluster'

# Define compute configuration
compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2',
                                                       min_nodes=0, max_nodes=4,
                                                       vm_priority='dedicated')

# Create the compute
aml_cluster = ComputeTarget.create(ws, compute_name, compute_config)
aml_cluster.wait_for_completion(show_output=True)

# Anexar um destino de computação não gerenciado com o SDK

In [ ]:
from azureml.core import Workspace
from azureml.core.compute import ComputeTarget, DatabricksCompute

# Load the workspace from the saved config file
ws = Workspace.from_config()

# Specify a name for the compute (unique within the workspace)
compute_name = 'db_cluster'

# Define configuration for existing Azure Databricks cluster
db_workspace_name = 'db_workspace'
db_resource_group = 'db_resource_group'
db_access_token = '1234-abc-5678-defg-90...'
db_config = DatabricksCompute.attach_configuration(resource_group=db_resource_group,
                                                   workspace_name=db_workspace_name,
                                                   access_token=db_access_token)

# Create the compute
databricks_compute = ComputeTarget.attach(ws, compute_name, db_config)
databricks_compute.wait_for_completion(True)

# Verificando um alvo de computação existente

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

compute_name = "aml-cluster"

# Check if the compute target exists
try:
    aml_cluster = ComputeTarget(workspace=ws, name=compute_name)
    print('Found existing cluster.')
except ComputeTargetException:
    # If not, create it
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2',
                                                           max_nodes=4)
    aml_cluster = ComputeTarget.create(ws, compute_name, compute_config)

aml_cluster.wait_for_completion(show_output=True)

# Use alvos de computação

In [ ]:
from azureml.core import Environment
from azureml.train.estimator import Estimator

compute_name = 'aml-cluster'

training_env = Environment.get(workspace=ws, name='training_environment')

estimator = Estimator(source_directory='experiment_folder',
                      entry_script='training_script.py',
                      environment_definition=training_env,
                      compute_target=compute_name)

Quando um experimento para o estimador é enviado, a execução é enfileirada enquanto o destino de computação é iniciado e o ambiente especificado é implantado nele, e então a execução é processada no ambiente de computação.

Em vez de especificar o nome do destino de computação, você pode especificar um objeto ComputeTarget , como este:

In [ ]:
from azureml.core import Environment
from azureml.train.estimator import Estimator
from azureml.core.compute import ComputeTarget

compute_name = 'aml-cluster'

training_cluster = ComputeTarget(workspace=ws, name=compute_name)

training_env = Environment.get(workspace=ws, name='training_environment')

estimator = Estimator(source_directory='experiment_folder',
                      entry_script='training_script.py',
                      environment_definition=training_env,
                      compute_target=training_cluster)